In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
# Get current script directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Get parent directory
parent_dir = os.path.dirname(current_dir)
# Add parent directory to sys.path
sys.path.append(parent_dir)

In [3]:
n = 100
p_median = 10

data = pd.read_csv(
    f"data/paper_data/GKD_data_{n}.txt",
    sep=r"\s+",
    header=None,
)

In [4]:
from src.data_class import ProblemData
problem_data = ProblemData(data=data)

In [5]:
problem_data.distance_matrix

array([[ 0.     , 60.89527, 79.96262, ..., 95.37163, 65.08042,  9.50322],
       [60.89527,  0.     , 83.12209, ..., 45.91741, 12.72628, 64.86382],
       [79.96262, 83.12209,  0.     , ..., 72.56293, 73.55716, 72.9181 ],
       ...,
       [95.37163, 45.91741, 72.56293, ...,  0.     , 34.47918, 95.46816],
       [65.08042, 12.72628, 73.55716, ..., 34.47918,  0.     , 67.3406 ],
       [ 9.50322, 64.86382, 72.9181 , ..., 95.46816, 67.3406 ,  0.     ]])

In [6]:
# Reload the updated class with numpy support
import importlib
import src.data_class
importlib.reload(src.data_class)
from src.data_class import ProblemData

# Create ProblemData with numpy distance matrix
problem_data = ProblemData(data=data)
print(f"Number of facilities: {problem_data.number_of_facilities}")
print(f"Distance matrix type: {type(problem_data.distance_matrix)}")
print(f"Distance matrix shape: {problem_data.distance_matrix.shape}")
print(f"Distance matrix dtype: {problem_data.distance_matrix.dtype}")

# Show a small portion of the matrix
print("\nFirst 5x5 of distance matrix:")
print(problem_data.distance_matrix[:5, :5])

# Test numpy operations
print("\nMatrix operations:")
print(f"Min distance (excluding zeros): {problem_data.distance_matrix[problem_data.distance_matrix > 0].min():.2f}")
print(f"Max distance: {problem_data.distance_matrix.max():.2f}")
print(f"Mean distance: {problem_data.distance_matrix[problem_data.distance_matrix > 0].mean():.2f}")

# Check if matrix is symmetric
is_symmetric = np.allclose(problem_data.distance_matrix, problem_data.distance_matrix.T)
print(f"Is symmetric: {is_symmetric}")

Number of facilities: 100
Distance matrix type: <class 'numpy.ndarray'>
Distance matrix shape: (100, 100)
Distance matrix dtype: float64

First 5x5 of distance matrix:
[[  0.       60.89527  79.96262  56.27752 102.63849]
 [ 60.89527   0.       83.12209  22.11952  62.64041]
 [ 79.96262  83.12209   0.       61.02421  61.10463]
 [ 56.27752  22.11952  61.02421   0.       49.48181]
 [102.63849  62.64041  61.10463  49.48181   0.     ]]

Matrix operations:
Min distance (excluding zeros): 0.72
Max distance: 135.26
Mean distance: 52.79
Is symmetric: True


In [7]:
# Benefits of numpy arrays for distance matrices:

# 1. Memory efficiency - numpy arrays use less memory than lists of lists
print("Memory and performance benefits:")
print(f"Matrix shape: {problem_data.distance_matrix.shape}")
print(f"Memory usage: {problem_data.distance_matrix.nbytes} bytes")

# 2. Fast mathematical operations
print("\nMathematical operations:")
# Find closest facility to facility 0
distances_from_0 = problem_data.distance_matrix[0, :]
distances_from_0_nonzero = distances_from_0[distances_from_0 > 0]
closest_to_0 = np.argmin(distances_from_0_nonzero)
print(f"Closest facility to facility 0: {np.where(distances_from_0 == distances_from_0_nonzero[closest_to_0])[0][0]}")
print(f"Distance: {distances_from_0_nonzero[closest_to_0]:.2f}")

# 3. Easy slicing and indexing
print("\nDistances from first 3 facilities to first 3 facilities:")
print(problem_data.distance_matrix[:3, :3])

# 4. Built-in statistical functions
print("\nStatistics:")
print(f"Standard deviation: {np.std(problem_data.distance_matrix[problem_data.distance_matrix > 0]):.2f}")
print(f"Median distance: {np.median(problem_data.distance_matrix[problem_data.distance_matrix > 0]):.2f}")

# 5. Easy to use with optimization libraries (scipy, numpy, etc.)
print("\nReady for optimization algorithms that expect numpy arrays!")

Memory and performance benefits:
Matrix shape: (100, 100)
Memory usage: 80000 bytes

Mathematical operations:
Closest facility to facility 0: 45
Distance: 4.47

Distances from first 3 facilities to first 3 facilities:
[[ 0.      60.89527 79.96262]
 [60.89527  0.      83.12209]
 [79.96262 83.12209  0.     ]]

Statistics:
Standard deviation: 25.13
Median distance: 52.26

Ready for optimization algorithms that expect numpy arrays!


In [8]:
from src.maxmin_diversity.bisection import BisectionMethod

solver = BisectionMethod(ratio=0.3)
_,results = solver.optimise(problem=problem_data, p_median=p_median)

print(results)
min_distance = results["optimal_distance"].iloc[0]

                                    algorithm  p_median  optimal_distance  \
0  Adaptive Bisection Method for P-Dispersion        10          34.11047   

   runtime_seconds  iterations  ratio   status  
0          0.63825           7    0.3  optimal  


In [9]:
# Reload the beta model module to get the latest changes
import importlib
import src.p_median.beta_model
from src.p_median.beta_model import BetaPmeanModel
import src.p_median.bender_decomposition   # adjust to the actual module path
import src.p_median.benders_master         # if quadratic_model lives here
import src.bi_objective.epsilon_method
importlib.reload(src.p_median.beta_model)
importlib.reload(src.p_median.bender_decomposition)
importlib.reload(src.p_median.benders_master)
importlib.reload(src.bi_objective.epsilon_method)

from src.p_median.bender_decomposition import BendersDecompositionModel
from src.p_median.benders_master import BendersMaster  # if needed
from src.bi_objective.epsilon_method import BiObjectiveEpsilonMethod



In [10]:
# Create BetaPmeanModel with correct parameter order
# Constructor signature: __init__(self, p_median: int, min_distance: float, problem_data: ProblemData, **data)
solver = BetaPmeanModel(p_median=p_median, min_distance=min_distance, problem_data=problem_data)

print("Solver created successfully!")
print(f"Number of facilities: {solver.number_of_locations}")
print(f"p_median: {solver.p_median}")
print(f"min_distance: {solver.min_distance}")

# Solve with beta constraint
result = solver.quadratic_model(beta=0, epsilon=1)
print("\nResult:", result)

Solver created successfully!
Number of facilities: 100
p_median: 10
min_distance: 34.11047

Result: (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 1273.7414899999999, 1.4833769798278809, 5756.849620000001, 'feasible')


In [11]:
solver = BendersDecompositionModel(problem_data=problem_data, p_median=p_median, min_distance=min_distance)
result = solver.branch_and_cut_benders(beta=0, epsilon=1, cutting_plane=True)
result

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_TimeLimit                               3600
Legacy callback                                  LD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Clique table members: 1299.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (1.53 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

      0     0        0.0000    17                      0.0000       25      

(array([ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0.,
         0.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  1.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 1273.74149,
 1273.74149,
 3.295823574066162,
 5756.849619999994)

In [12]:
solver.balinski_cut(beta=0, epsilon=1, cutting_plane=True)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 1273.7414899999999,
 0.5492656230926514,
 5756.849620000001,
 'feasible')

In [13]:
optimizer = BiObjectiveEpsilonMethod(problem_data=problem_data, p_median=p_median, min_distance=min_distance)

In [14]:
results = optimizer.epsilon_method(
    method="quadratic",
    epsilon=1,
    cutting_plane=True
)
results


No feasible solution found for beta=6272.854100
Error in quadratic model solution: '>' not supported between instances of 'NoneType' and 'float'


{1: {'solution_vector': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'objective_value': 1273.7414899999999,
  'achieved_beta': 5756.849620000001,
  'iter_runtime': 2.063948154449463,
  'method': 'quadratic',
  'iteration': 1,
  'beta_target': 0.0,
  'selected_facilities': [11, 22, 27, 50, 55, 69, 70, 77, 84, 89],
  'num_pareto_solutions': 1,
  'total_runtime': 2.063948154449463},
 2: {'solution_vector': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0

In [16]:
results = optimizer.epsilon_method(
    method="cutting_plane",
    epsilon=1,
    cutting_plane=True
)
results

Error in cutting plane solution: '>' not supported between instances of 'NoneType' and 'float'


{1: {'solution_vector': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'objective_value': 1273.7414899999999,
  'achieved_beta': 5756.849620000001,
  'iter_runtime': 1.3642878532409668,
  'method': 'cutting_plane',
  'iteration': 1,
  'beta_target': 0.0,
  'selected_facilities': [11, 22, 27, 50, 55, 69, 70, 77, 84, 89],
  'num_pareto_solutions': 1,
  'total_runtime': 1.3642878532409668},
 2: {'solution_vector': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.,

In [17]:
results = optimizer.epsilon_method(
    method="benders",
    epsilon=1,
    cutting_plane=True
)
results

Error in Benders decomposition: zero-size array to reduction operation minimum which has no identity


{1: {'solution_vector': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'objective_value': 1273.7414899999999,
  'achieved_beta': 5756.849620000001,
  'iter_runtime': 0.7798421382904053,
  'method': 'benders',
  'iteration': 1,
  'beta_target': 0.0,
  'selected_facilities': [11, 22, 27, 50, 55, 69, 70, 77, 84, 89],
  'num_pareto_solutions': 1,
  'total_runtime': 0.7798421382904053},
 2: {'solution_vector': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0

In [18]:
results = optimizer.epsilon_method(
    method="branch_cut_benders",
    epsilon=1,
    cutting_plane=True
)
results

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_TimeLimit                               3600
Legacy callback                                  LD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Clique table members: 1299.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.01 sec. (1.53 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

      0     0        0.0000    17                      0.0000       25      

Branch-and-cut found no solution. Status: 103


{1: {'solution_vector': array([ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
         -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,
          0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
          0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0.,
          0.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         -0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  1.,  0.,
          0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  'objective_value': 1273.74149,
  'achieved_beta': 5756.849619999994,
  'iter_runtime': 4.471685171127319,
  'method': 'branch_cut_benders',
  'iteration': 1,
  'beta_target': 0.0,
  'selected_facilities': [11, 22, 27, 50, 55, 69, 70, 77, 84, 89],
  'num_pareto_solutions': 1,
  'total_runtime': 4.471685171127319},
 2: {'solution_vector': array([ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  

In [ ]:

filename = optimizer.export_pareto_solutions(solutions_dict=results)
print(filename)